In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
df_mort= pd.read_table('mortality.txt',delimiter=';',names=['Cancer_code', 'Sex', 'Age_bucket', 'Location', 'Year', 'Nb_deaths'],low_memory=False)
df_mort.describe()

,Sex,Age_bucket,Year,Nb_deaths
count,6.359642e+06,6.359642e+06,6.359642e+06,6.359642e+06
mean,1.540061e+00,1.000000e+01,1.992326e+03,3.230369e+01
std,6.528709e-01,5.477226e+00,1.609652e+01,2.649799e+02
min,1.000000e+00,1.000000e+00,1.950000e+03,0.000000e+00
25%,1.000000e+00,5.000000e+00,1.981000e+03,0.000000e+00
50%,2.000000e+00,1.000000e+01,1.996000e+03,0.000000e+00
75%,2.000000e+00,1.500000e+01,2.006000e+03,5.000000e+00
max,9.000000e+00,1.900000e+01,2.015000e+03,3.493500e+04


- Aggregation de la tranche d'age 16 et suppression des tranches d'age 17,18 et 19

In [3]:
length=len(df_mort[df_mort['Age_bucket']==16])
index=df_mort[df_mort['Age_bucket']==16]['Nb_deaths'].index
j=list(range(0, length, 1))
t_16=df_mort[df_mort['Age_bucket']==16]['Nb_deaths'].values
t_17=df_mort[df_mort['Age_bucket']==17]['Nb_deaths'].values
t_18=df_mort[df_mort['Age_bucket']==18]['Nb_deaths'].values
t_19=df_mort[df_mort['Age_bucket']==19]['Nb_deaths'].values
df_mort['Nb_deaths'][index]=t_16[j] + t_17[j] + t_18[j] + t_19[j]
index17_18_19=df_mort[(df_mort['Age_bucket']==17) | (df_mort['Age_bucket']==18) | (df_mort['Age_bucket']==19)]['Nb_deaths'].index
df_mort=df_mort.drop(df_mort.index[index17_18_19])

C:\Users\DIARRA née CISSE Bal\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [4]:
#Attention il faut modifier la derniere ligne
df_age = pd.read_csv('age.csv', header = None , sep=' ')
df_age.columns=['Age_bucket', 'Age']
df_age.head()

,Age_bucket,Age
0,1,0-4
1,2,5-9
2,3,10-14
3,4,15-19
4,5,20-24


In [5]:
#merge age class df + main df
df1 = pd.merge(df_mort, df_age, how='left', left_on='Age_bucket', right_on='Age_bucket')
df1.head()

,Cancer_code,Sex,Age_bucket,Location,Year,Nb_deaths,Age
0,C80,2,1,Jamaica,2000,2,0-4
1,C80,2,2,Jamaica,2000,0,5-9
2,C80,2,3,Jamaica,2000,1,10-14
3,C80,2,4,Jamaica,2000,0,15-19
4,C80,2,5,Jamaica,2000,0,20-24


In [6]:
#read csv file - WorldBank indicators
df_wb = pd.read_csv('WorldBank_Data.csv', sep=',')

In [7]:
#•Harmonisation des noms des pays entre les deux fichiers "mortality" et "worldbank"
df_correspondance=pd.read_table('correspondance.txt',delimiter=';')
df_correspondance.head()
Freq_area2 = df_correspondance.groupby(['area2'])['area2'].count()
# on crée une nouvelle colonne "area2" dane la base worldbank qui correspond au nom du pays dans la base mortality
df_wb2 = pd.merge(df_wb, df_correspondance, on='area')
df_wb2['area'].describe() #258 pays différents
df_wb2['area2'].describe() #131 pays différents

count     11648
unique      131
top            
freq       5663
Name: area2, dtype: object

In [8]:
#calculate new variables : transform percentage into absolute values for each group
#naming convention = old name without the .5Y at the end and without the SP in the beginning
pop_par_tranche_dage=['POP.0004.','POP.0509.','POP.1014.','POP.1519.','POP.2024.','POP.2529.','POP.3034.','POP.3539.','POP.4044.','POP.4549.','POP.5054.','POP.5559.','POP.6064.','POP.6569.','POP.7074.','POP.7579.','POP.80UP.']
for i in range(0,17):
    df_wb2[''+pop_par_tranche_dage[i]+'FE'] =(df_wb2['SP.POP.TOTL']*df_wb2['SP.POP.TOTL.FE.ZS']*df_wb2['SP.'+pop_par_tranche_dage[i]+'FE.5Y'])/10000
    df_wb2[''+pop_par_tranche_dage[i]+'MA'] =(df_wb2['SP.POP.TOTL']*(100-df_wb2['SP.POP.TOTL.FE.ZS'])*df_wb2['SP.'+pop_par_tranche_dage[i]+'MA.5Y'])/10000


In [9]:
## subsetting dataframe with columns to merge
df_mini = df_wb2[['area','year','POP.0004.FE', 'POP.0509.FE', 'POP.1014.FE', 'POP.1519.FE', 'POP.2024.FE', 'POP.2529.FE', 'POP.3034.FE', 'POP.3539.FE', 'POP.4044.FE', 'POP.4549.FE', 'POP.5054.FE', 'POP.5559.FE', 'POP.6064.FE', 'POP.6569.FE', 'POP.7074.FE', 'POP.7579.FE', 'POP.80UP.FE', 'POP.0004.MA', 'POP.0509.MA', 'POP.1014.MA', 'POP.1519.MA', 'POP.2024.MA', 'POP.2529.MA', 'POP.3034.MA', 'POP.3539.MA', 'POP.4044.MA', 'POP.4549.MA', 'POP.5054.MA', 'POP.5559.MA', 'POP.6064.MA', 'POP.6569.MA', 'POP.7074.MA', 'POP.7579.MA', 'POP.80UP.MA']]
df_mini.head()
#list of variable names about sex= female / male
df_mini.columns[df_mini.columns.str.endswith('FE')]
df_mini.columns[df_mini.columns.str.endswith('MA')]

Index(['POP.0004.MA', 'POP.0509.MA', 'POP.1014.MA', 'POP.1519.MA',
       'POP.2024.MA', 'POP.2529.MA', 'POP.3034.MA', 'POP.3539.MA',
       'POP.4044.MA', 'POP.4549.MA', 'POP.5054.MA', 'POP.5559.MA',
       'POP.6064.MA', 'POP.6569.MA', 'POP.7074.MA', 'POP.7579.MA',
       'POP.80UP.MA'],
      dtype='object')

In [10]:
#reshaping
df_mini.set_index(['area', 'year'], inplace=True)
df_mini.columns = pd.MultiIndex.from_tuples(tuple(df_mini.columns.str.split(".")))

transf = df_mini.stack(level = [1,2]).reset_index()
transf.columns = ['Country', 'Year', 'Age_group', 'Sex', 'Population']

In [11]:
#match labels
'''
MA/FE -> 1/2
0004/0509/1014... -> 1/2/3...
!!!classes in worldbank folder cap at 80, mortality folder has more classes(17:80-84, 18:85+, 19:unknown)
'''
transf['Sex'].replace(['MA', 'FE'],[1,2], inplace=True)
transf['Age_group'].replace(['0004','0509','1014','1519','2024','2529','3034','3539','4044','4549','5054','5559','6064','6569','7074','7579'],
list(range(1, 17)), inplace=True)

In [12]:
a = pd.merge(df_mort[df_mort['Year'] >=2000  ] , 
              transf[transf['Year']>=2000 ] , 
              how='right', 
              left_on=['Location', 'Year', 'Age_bucket', 'Sex'], 
right_on=['Country', 'Year', 'Age_group', 'Sex'])
a.head(5)

,Cancer_code,Sex,Age_bucket,Location,Year,Nb_deaths,Country,Age_group,Population
0,C80,2,1,Jamaica,2000,2.0,Jamaica,1,139549.999999
1,C56,2,1,Jamaica,2000,0.0,Jamaica,1,139549.999999
2,C64,2,1,Jamaica,2000,1.0,Jamaica,1,139549.999999
3,C44,2,1,Jamaica,2000,0.0,Jamaica,1,139549.999999
4,C84,2,1,Jamaica,2000,0.0,Jamaica,1,139549.999999


In [13]:
a.drop('Cancer_code', axis=1, inplace=True)
a.drop('Age_bucket', axis=1, inplace=True)
a.drop('Location', axis=1, inplace=True)
a.drop('Nb_deaths', axis=1, inplace=True)
a.head()

,Sex,Year,Country,Age_group,Population
0,2,2000,Jamaica,1,139549.999999
1,2,2000,Jamaica,1,139549.999999
2,2,2000,Jamaica,1,139549.999999
3,2,2000,Jamaica,1,139549.999999
4,2,2000,Jamaica,1,139549.999999


In [14]:
indicators = ['area','year','SP.URB.TOTL.IN.ZS',	'SP.URB.GROW',	'SP.POP.GROW',	'SE.PRM.DURS',	'AG.SRF.TOTL.K2',	'SE.PRE.DURS',	'EN.POP.DNST',	'SE.SEC.DURS',	'SP.DYN.CBRT.IN',	'SP.DYN.CDRT.IN',	'NY.ADJ.AEDU.GN.ZS',	'SP.DYN.TFRT.IN',	'SP.ADO.TFRT',	'SP.DYN.LE00.FE.IN',	'SP.DYN.LE00.IN',	'SP.POP.DPND',	'SP.DYN.AMRT.FE',	'SP.DYN.AMRT.MA',	'EN.ATM.NOXE.KT.CE',	'EN.ATM.METH.KT.CE',	'AG.LND.AGRI.K2',	'EN.ATM.GHGO.KT.CE',	'AG.LND.AGRI.ZS',	'AG.PRD.FOOD.XD',	'IT.CEL.SETS',	'AG.PRD.LVSK.XD',	'AG.PRD.CROP.XD',	'IT.MLT.MAIN',	'EN.ATM.GHGT.KT.CE',	'EN.ATM.CO2E.GF.ZS',	'EN.ATM.CO2E.LF.ZS',	'EN.ATM.CO2E.SF.ZS',	'NY.GDP.FRST.RT.ZS',	'NY.GDP.MINR.RT.ZS',	'EN.ATM.CO2E.LF.KT',	'EN.ATM.METH.AG.KT.CE',	'EN.ATM.METH.EG.KT.CE',	'EN.ATM.NOXE.AG.KT.CE',	'EN.ATM.NOXE.EG.KT.CE',	'NY.GDP.TOTL.RT.ZS',	'EN.ATM.CO2E.KT',	'EN.ATM.CO2E.PC',	'NY.GDP.MKTP.CD',	'NY.GDP.PCAP.CD']
b=pd.merge(a, 
              df_wb2.loc[:,indicators], 
              how='left',
              left_on=['Country','Year'], 
         right_on=['area','year'])

In [15]:
b.drop('area', axis=1, inplace=True)
b.drop('year', axis=1, inplace=True)
b.head()

,Sex,Year,Country,Age_group,Population,SP.URB.TOTL.IN.ZS,SP.URB.GROW,SP.POP.GROW,SE.PRM.DURS,AG.SRF.TOTL.K2,...,EN.ATM.CO2E.LF.KT,EN.ATM.METH.AG.KT.CE,EN.ATM.METH.EG.KT.CE,EN.ATM.NOXE.AG.KT.CE,EN.ATM.NOXE.EG.KT.CE,NY.GDP.TOTL.RT.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD
0,2,2000,Jamaica,1,139549.999999,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
1,2,2000,Jamaica,1,139549.999999,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
2,2,2000,Jamaica,1,139549.999999,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
3,2,2000,Jamaica,1,139549.999999,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
4,2,2000,Jamaica,1,139549.999999,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741


In [16]:
b=b.groupby('Country').apply(lambda x: (x.sort_values('Year', ascending=True)))
b.fillna(method='ffill',inplace=True)
b.fillna(method='bfill',inplace=True)
b.reset_index(drop=True,inplace=True)

In [18]:
len(b)

1845548

In [ ]:
import tempfile
tempfile.tempdir='C:/Temp/'
with tempfile.NamedTemporaryFile() as temp:

    b.to_csv(temp.name + '.csv')